<a href="https://colab.research.google.com/github/SanePai/Neural_Networks-Assignment-2/blob/main/Question_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%cd "/content/drive/MyDrive/NNFL/Assignment 2"
!ls

/content/drive/.shortcut-targets-by-id/104/Assignment 2
assignment2.pdf  class2_images	class_label.mat  data5.mat     input.mat
class1_images	 class3_images	data55.xlsx	 input_a2.mat  label.mat


In [ ]:
import numpy as np
from scipy.io import loadmat
from sklearn.preprocessing import normalize
import random
from sklearn.metrics import confusion_matrix as cm

In [ ]:
#Load data, shuffle and normalize
mat_contents = loadmat('data5.mat')
data = mat_contents['x']
np.random.shuffle(data)

In [ ]:
def init_data():
    X = np.array(data[ : , :-1], dtype = float)
    y = np.array(data[ : , -1], dtype = int)
    X = normalize(X, axis = 0)
    return X, y


X, y_ = init_data()
y = np.zeros((len(y_), 2))
for i in range(len(y_)):
    if y_[i]==1:
        y[i,1] = 1.0
    elif y_[i]==0:
        y[i,0] = 1.0


X_train, y_train = X[ :int(0.7 * len(X))], y[ :int(0.7 * len(X))]
X_val, y_val = X[ int(0.7 * len(X)): ], y[ int(0.7 * len(X)): ]

alpha = 0.5

In [ ]:
#Sigmoid function
def sigmoid(x, derivative=False):
        if (derivative == True):
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

#Tanh function
def tanh(x):
    return np.tanh(x)

#Cost function
def calc_cost(NN,x ,y):
    
    cost = 0
    for i in range(len(x)):
        x_ = np.reshape(x[i], (len(x[i]), 1))
        cost += 0.5 / len(x) * np.sum((y[i] - NN.forward_pass(x_)) ** 2)
    
    return cost

In [ ]:
#Neural network class
class NeuralNetwork(object):
    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.W = {}
        self.a = {}
        self.b = {}
        
        #Initialize Weights
        for i in range(1, self.num_layers):
            self.W[i] = np.random.randn(self.sizes[i-1], self.sizes[i])
            
        #Initialize biases
        for i in range(1, self.num_layers):
            self.b[i] = np.random.randn(self.sizes[i], 1)
        
        #Initialize activations
        for i in range(1, self.num_layers):
            self.a[i] = np.zeros([self.sizes[i], 1])
        
    #Forward pass to compute scores
    def forward_pass(self, X):
        
        self.a[0] = X
        
        for i in range(1, self.num_layers):
            self.a[i] = sigmoid(np.dot(self.W[i].T, self.a[i-1]) + self.b[i])

        return self.a[self.num_layers-1] 
    
    #Backward pass to update weights
    def backward_pass(self, X, Y, output):
        
        self.d = {}
        self.d_output = (Y - output) * sigmoid(output, derivative=True)
        self.d[self.num_layers-1] = self.d_output
        
        #Derivatives of the layers
        for i in range(self.num_layers-1, 1, -1):
            self.d[i-1] = np.dot(self.W[i], self.d[i]) * sigmoid(self.a[i-1], derivative=True)
        
        #Updating weights
        for i in range(1, self.num_layers-1):
            self.W[i] += alpha * np.dot(self.a[i-1], self.d[i].T)
            
        #Updating biases
        for i in range(1, self.num_layers-1):
            self.b[i] += alpha * self.d[i]

    #Training helper function   
    def train(self, X, Y):
        X = np.reshape(X, (len(X), 1))
        output = self.forward_pass(X)
        self.backward_pass(X, Y, output)

    #Get weights    
    def get_W(self):
        return self.W
    
    #Load specified weights
    def load_W(self, W):
        self.W = W

    #Scores computation for given input    
    def get_a(self, x):
        x = np.reshape(x, (len(x), 1))
        self.forward_pass(x)
        return self.a
    
    #Helper function for autoencoder chaining
    def load_a(self, a):
        self.a = a

In [ ]:
#Initialization
autoencoder1 = NeuralNetwork([72, 60, 72])
autoencoder2 = NeuralNetwork([60,40,60])
print("Autoencoder 1 Pre-Training")
#Autoencoder 1 pre-training
for i in range(500):
    for j, row in enumerate(X_train):
        row = np.reshape(row, (72,1))
        autoencoder1.train(row, row)
        
    cost = calc_cost(autoencoder1, X_train, X_train)
    if (i+1)%100 == 0:
        print(f"Epoch {i+1}, Cost {cost}")
    
autoencoder2_input = []

for row in X_train:
    autoencoder2_input.append(autoencoder1.get_a(row)[1])

autoencoder2_input = np.array(autoencoder2_input)
print("===========================")
print("Autoencoder 2 Pre-Training")
#Autoencoder 2 pre-training
for i in range(500):
    for j, row in enumerate(autoencoder2_input):
        row = np.reshape(row, (60,1))
        autoencoder2.train(row, row)
        
    cost = calc_cost(autoencoder2, autoencoder2_input, autoencoder2_input)
    if (i+1)%100 == 0:
        print(f"Epoch {i+1}, Cost {cost}")

Autoencoder 1 Pre-Training
Epoch 100, Cost 278.7576880590857
Epoch 200, Cost 278.72640314691347
Epoch 300, Cost 278.7191837749819
Epoch 400, Cost 278.71675215075226
Epoch 500, Cost 278.7157875197329
Autoencoder 2 Pre-Training
Epoch 100, Cost 3.442753475860261
Epoch 200, Cost 3.4420245000129333
Epoch 300, Cost 3.441885435489334
Epoch 400, Cost 3.441762577770607
Epoch 500, Cost 3.4416721655843308


In [ ]:
#ELM Input
elm_input = []
for row in autoencoder2_input:
    elm_input.append(autoencoder2.get_a(row)[1])   
elm_input = np.array(elm_input)

#ELM Params
elm_neurons = 300
output_neurons = 2
W_elm = np.random.randn(elm_input.shape[1], elm_neurons)

#ELM Training
np.random.seed(1)
elm_input = np.reshape(elm_input, (1503, 40))
H = np.matmul(elm_input, W_elm)
H = tanh(H)
H_inv = np.linalg.pinv(H)
W_final = np.matmul(H_inv, y_train) 

In [ ]:
#Testing on validation set

#Autoencoder 1 forward pass
layer1_out = []

for i, row in enumerate(X_val):
    act = autoencoder1.get_a(row)[1]
    layer1_out.append(act)
    
layer1_out = np.array(layer1_out)
layer1_out = np.reshape(layer1_out, (645, 60))

#Autoencoder 2 forward pass
layer2_out = []

for i, row in enumerate(layer1_out):
    act = autoencoder2.get_a(row)[1]
    layer2_out.append(act)
    
layer2_out = np.array(layer2_out)
layer2_out = np.reshape(layer2_out, (645, 40))

#ELM forward pass
H_T = np.matmul(layer2_out, W_elm)
H_T = tanh(H_T)
y_pred = np.matmul(H_T, W_final)

In [ ]:
a = [np.argmax(y_pred[i]) for i in range(len(y_pred))]
b = [np.argmax(y_val[i]) for i in range(len(y_val))]

confmat = cm(b,a)

Accuracy = (confmat[0][0]+confmat[1][1])/len(y_pred)
Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
print("Confusion Matrix:")
print(confmat)
print("\n")
print(f"Accuracy: {Accuracy*100}%\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}\n")

Confusion Matrix:
[[283  32]
 [ 47 283]]


Accuracy: 87.75193798449612%
Sensitivity: 0.8575757575757575
Specificity: 0.8984126984126984

